# Gemini Provider Test

**IMPORTANT**: Run cells in order, or use "Restart Kernel and Run All"

In [1]:
import sys
sys.path.insert(0, '..')
from backend.rag_service.core.rag_engine import RAGEngine, RAGConfig

In [2]:
# Test if settings are loading correctly
from backend.rag_service.config.settings import settings

print("Gemini API Key loaded:", settings.gemini.api_key[:20] + "..." if settings.gemini.api_key else "NOT LOADED")
print("Gemini Model:", settings.gemini.model)
print()
print("Full Gemini Settings:")
print(f"  API Key: {settings.gemini.api_key[:20] + '...' if settings.gemini.api_key else 'None'}")
print(f"  Model: {settings.gemini.model}")
print(f"  Max Tokens: {settings.gemini.max_tokens}")
print(f"  Temperature: {settings.gemini.temperature}")

Gemini API Key loaded: AIzaSyDi02qEcFXZFvR9...
Gemini Model: gemini-2.5-flash

Full Gemini Settings:
  API Key: AIzaSyDi02qEcFXZFvR9...
  Model: gemini-2.5-flash
  Max Tokens: 8192
  Temperature: 0.3


In [3]:
# Force reload settings module
import importlib
import sys
if 'backend.rag_service.config.settings' in sys.modules:
    del sys.modules['backend.rag_service.config.settings']

from backend.rag_service.config.settings import settings

print("Gemini API Key loaded:", settings.gemini.api_key[:20] + "..." if settings.gemini.api_key else "NOT LOADED")
print("Gemini Model:", settings.gemini.model)
print()
print("Full Gemini Settings:")
print(f"  API Key: {settings.gemini.api_key[:20] + '...' if settings.gemini.api_key else 'None'}")
print(f"  Model: {settings.gemini.model}")
print(f"  Max Tokens: {settings.gemini.max_tokens}")
print(f"  Temperature: {settings.gemini.temperature}")
print()
print("LM Studio Settings:")
print(f"  Base URL: {settings.lmstudio.base_url}")
print(f"  Model: {settings.lmstudio.model}")

Gemini API Key loaded: AIzaSyDi02qEcFXZFvR9...
Gemini Model: gemini-2.5-flash

Full Gemini Settings:
  API Key: AIzaSyDi02qEcFXZFvR9...
  Model: gemini-2.5-flash
  Max Tokens: 8192
  Temperature: 0.3

LM Studio Settings:
  Base URL: http://localhost:1234/v1
  Model: local-model


In [4]:
# Test Qdrant connection directly
from qdrant_client import QdrantClient

print(f"Qdrant settings from config:")
print(f"  Host: {settings.qdrant.host}")
print(f"  Port: {settings.qdrant.port}")
print(f"  Prefer gRPC: {settings.qdrant.prefer_grpc}")
print()

# Try connecting with proper URL format
try:
    client = QdrantClient(host=settings.qdrant.host, port=settings.qdrant.port, prefer_grpc=False, https=False)
    collections = client.get_collections()
    print(f"✓ Connected successfully! Collections: {[c.name for c in collections.collections]}")
except Exception as e:
    print(f"✗ Connection failed: {e}")

Qdrant settings from config:
  Host: localhost
  Port: 6333
  Prefer gRPC: False

✓ Connected successfully! Collections: ['fatwas', 'hadiths']


In [5]:
# Test FULL linked retrieval pipeline (fatwas + hadiths)
print("=" * 70)
print("Testing Linked Retrieval: Fatwas + Referenced Hadiths")
print("=" * 70)

question = "ما حكم الزكاة"
print(f"\nQuery: {question}\n")

# Configure RAG with linked retrieval enabled
config = RAGConfig(
    top_k=3,  # 3 fatwas
    use_linked_retrieval=True,  # Enable hadith linking!
    hadith_collection="hadiths",
    max_hadiths_per_fatwa=2,  # Up to 2 hadiths per fatwa
    include_citations=True,
    citation_format="islamic_scholarly",
    max_context_tokens=2000,
)

# Create engine
engine = RAGEngine(config=config, provider_name="gemini")

print("=" * 70)
print("Step 1: Retrieving documents with linked hadiths...")
print("=" * 70)

# Manually retrieve to inspect what's happening
if engine.linked_retriever:
    linked_result = engine.linked_retriever.retrieve_with_links(
        query=question,
        top_k_fatwas=3,
        top_k_hadiths_per_fatwa=2,
    )
    
    print(f"\n✓ Retrieved {len(linked_result.primary_docs)} fatwas")
    print(f"✓ Retrieved {len(linked_result.linked_hadiths)} linked hadiths")
    print(f"✓ Total documents for context: {linked_result.total_documents}\n")
    
    # Show fatwas
    print("-" * 70)
    print("FATWAS:")
    print("-" * 70)
    for i, fatwa in enumerate(linked_result.primary_docs, 1):
        print(f"\n[Fatwa {i}]")
        print(f"  Title: {fatwa.title[:100] if fatwa.title else fatwa.fatwa_id}")
        print(f"  Source: {fatwa.source}")
        print(f"  Score: {fatwa.score:.4f}")
        print(f"  Content preview: {fatwa.content[:200]}...")
    
    # Show linked hadiths
    if linked_result.linked_hadiths:
        print("\n" + "-" * 70)
        print("LINKED HADITHS:")
        print("-" * 70)
        for i, hadith in enumerate(linked_result.linked_hadiths, 1):
            print(f"\n[Hadith {i}]")
            print(f"  Source: {hadith.metadata.get('source', 'Unknown')}")
            print(f"  Narrator: {hadith.metadata.get('narrator', 'Unknown')}")
            print(f"  Score: {hadith.score:.4f}")
            print(f"  Content preview: {hadith.content[:150]}...")
    else:
        print("\n⚠️ No linked hadiths found (fatwas may not reference hadiths)")
    
    # Test context formatting
    print("\n" + "=" * 70)
    print("Step 2: Formatting context with mixed sources...")
    print("=" * 70)
    
    all_docs = linked_result.all_documents
    formatted_context = engine.context_manager.format_context(all_docs)
    
    print(f"\n✓ Context includes {len(formatted_context.documents_used)} documents")
    print(f"✓ Context length: {len(formatted_context.text)} chars (~{len(formatted_context.text) // 4} tokens)")
    print(f"✓ Truncated: {formatted_context.truncated}")
    
    # Show context preview
    print(f"\nContext preview (first 800 chars):")
    print("-" * 70)
    print(formatted_context.text[:800])
    print("...")
    
    # Test citation generation
    print("\n" + "=" * 70)
    print("Step 3: Testing citation generation...")
    print("=" * 70)
    
    citations = engine.citation_generator.generate_citations(formatted_context.documents_used)
    
    print(f"\n✓ Generated {len(citations)} citations\n")
    
    for i, citation in enumerate(citations, 1):
        print(f"[{i}] {citation.get('formatted', citation.get('source', 'Unknown'))}")
    
    # Show prompt details
    print("\n" + "=" * 70)
    print("Step 4: Final prompt details...")
    print("=" * 70)
    
    from backend.rag_service.utils.prompts import get_system_prompt, build_rag_prompt
    
    system_prompt = get_system_prompt("ar")
    user_message = build_rag_prompt(question, formatted_context.text, "ar")
    
    total_tokens = (len(system_prompt) + len(user_message)) // 4
    
    print(f"\n✓ System prompt: {len(system_prompt)} chars (~{len(system_prompt) // 4} tokens)")
    print(f"✓ User message: {len(user_message)} chars (~{len(user_message) // 4} tokens)")
    print(f"✓ Total estimated: ~{total_tokens} tokens")
    
    print("\n" + "=" * 70)
    print("✅ Pipeline ready! (Gemini quota exceeded - can't generate response)")
    print("=" * 70)
    print("\nTo test with LM Studio instead, change provider_name='lmstudio'")
    
else:
    print("❌ Linked retriever not initialized!")


Testing Linked Retrieval: Fatwas + Referenced Hadiths

Query: ما حكم الزكاة

Step 1: Retrieving documents with linked hadiths...


d:\_hsproject\nlp project\backend\..\backend\rag_service\core\retriever.py:105: UserWarning: Api key is used with an insecure connection.
  self._qdrant_client = QdrantClient(
2025-12-28 16:08:38.832 | INFO     | backend.rag_service.core.retriever:initialize:112 - Connected to Qdrant at localhost:6333
2025-12-28 16:08:55.300 | INFO     | backend.rag_service.core.retriever:initialize:123 - Loaded embedding model: Omartificial-Intelligence-Space/GATE-AraBert-v1
2025-12-28 16:08:59.109 | INFO     | backend.rag_service.core.retriever:initialize:134 - Loaded re-ranker: D:\_hsproject\nlp project\testing\model\Namaa-ARA-Reranker-V1
2025-12-28 16:08:59.914 | INFO     | backend.rag_service.core.linked_retrieval:retrieve_with_links:117 - Retrieved 3 fatwas for query



✓ Retrieved 3 fatwas
✓ Retrieved 0 linked hadiths
✓ Total documents for context: 3

----------------------------------------------------------------------
FATWAS:
----------------------------------------------------------------------

[Fatwa 1]
  Title: زكاة الأسهم المشتراة بقصد الانتفاع بأرباحها
  Source: إسلام ويب
  Score: 0.0333
  Content preview: السؤال: اشتريت أسهماً لبنك إسلامي وأنتفع بأرباح تلك الأسهم، فهل علي زكاة في هذه الأسهم؟.

الجواب: الحمد لله، والصلاة والسلام على رسول الله، وعلى آله وصحبه، أما بعد: فإن كان المقصود من الأسهم الانتفاع ...

[Fatwa 2]
  Title: زكاة من عنده مال وعليه ديون تستغرقه
  Source: إسلام ويب
  Score: 0.0208
  Content preview: السؤال: لديّ مبلغ من المال تجاوز نصاب الزكاة، وفي نفس الوقت عليّ ديون تتجاوز هذا المبلغ، ولكنها ديون نصف سنوية، أدفعها على أقساط كل ستة أشهر، وخوفًا من صرف هذه الأموال وضعتها في بنك إسلامي في شكل شهاد...

[Fatwa 3]
  Title: المال الذي تجب فيه الزكاة
  Source: إسلام ويب
  Score: 0.0144
  Content preview: السؤال: لدي مكتب خدمات عام

In [6]:
# Test with a query more likely to have hadith references
print("=" * 70)
print("Testing with query that typically has hadith references")
print("=" * 70)

question2 = "ما حكم صلاة الجماعة"  # Prayer in congregation - likely to have hadiths
print(f"\nQuery: {question2}\n")

config2 = RAGConfig(
    top_k=3,
    use_linked_retrieval=True,
    hadith_collection="hadiths",
    max_hadiths_per_fatwa=2,
    include_citations=True,
)

engine2 = RAGEngine(config=config2, provider_name="gemini")

if engine2.linked_retriever:
    linked_result2 = engine2.linked_retriever.retrieve_with_links(
        query=question2,
        top_k_fatwas=3,
        top_k_hadiths_per_fatwa=2,
    )
    
    print(f"✓ Retrieved {len(linked_result2.primary_docs)} fatwas")
    print(f"✓ Retrieved {len(linked_result2.linked_hadiths)} linked hadiths")
    print(f"✓ Total documents: {linked_result2.total_documents}\n")
    
    # Show fatwas briefly
    print("-" * 70)
    print("FATWAS:")
    for i, fatwa in enumerate(linked_result2.primary_docs, 1):
        print(f"  [{i}] {fatwa.title[:80]} (Score: {fatwa.score:.4f})")
    
    # Show linked hadiths
    if linked_result2.linked_hadiths:
        print("\n" + "-" * 70)
        print("✅ LINKED HADITHS FOUND:")
        print("-" * 70)
        for i, hadith in enumerate(linked_result2.linked_hadiths, 1):
            print(f"\n[Hadith {i}]")
            print(f"  Source: {hadith.metadata.get('source', 'Unknown')}")
            print(f"  Narrator: {hadith.metadata.get('narrator', 'Unknown')[:60]}...")
            print(f"  Book: {hadith.metadata.get('book', 'Unknown')}")
            print(f"  Number: {hadith.metadata.get('number', 'Unknown')}")
            print(f"  Score: {hadith.score:.4f}")
            print(f"  Arabic: {hadith.content[:150]}...")
            if hadith.metadata.get('english_translation'):
                print(f"  English: {hadith.metadata.get('english_translation')[:150]}...")
        
        # Test citations with mixed sources
        print("\n" + "=" * 70)
        print("CITATIONS (Fatwas + Hadiths):")
        print("=" * 70)
        
        all_docs = linked_result2.all_documents
        citations = engine2.citation_generator.generate_citations(all_docs)
        
        for i, citation in enumerate(citations, 1):
            print(f"\n[{i}] {citation.get('formatted', citation.get('source', 'Unknown'))}")
        
        print("\n" + "=" * 70)
        print("✅ Linked retrieval working! Hadiths found and citations generated.")
        print("=" * 70)
    else:
        print("\n⚠️ No hadiths found for this query either")
        print("Note: Reference extraction may need debugging")


Testing with query that typically has hadith references

Query: ما حكم صلاة الجماعة



d:\_hsproject\nlp project\backend\..\backend\rag_service\core\retriever.py:105: UserWarning: Api key is used with an insecure connection.
  self._qdrant_client = QdrantClient(
2025-12-28 16:11:13.089 | INFO     | backend.rag_service.core.retriever:initialize:112 - Connected to Qdrant at localhost:6333
'(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: 9867558e-c516-4bae-9839-65b6a187446e)')' thrown while requesting HEAD https://huggingface.co/Omartificial-Intelligence-Space/GATE-AraBert-v1/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].
2025-12-28 16:11:19.982 | INFO     | backend.rag_service.core.retriever:initialize:123 - Loaded embedding model: Omartificial-Intelligence-Space/GATE-AraBert-v1
2025-12-28 16:11:23.618 | INFO     | backend.rag_service.core.retriever:initialize:134 - Loaded re-ranker: D:\_hsproject\nlp project\testing\model\Namaa-ARA-Reranker-V1
2025-12-28 16:12:23.809 | INFO     | backend.ra

✓ Retrieved 3 fatwas
✓ Retrieved 0 linked hadiths
✓ Total documents: 3

----------------------------------------------------------------------
FATWAS:
  [1] من أحكام صلاة الجماعة (Score: 0.9691)
  [2] مسائل وتصويبات حول صلاة الجماعة (Score: 0.4845)
  [3] مذاهب العلماء في حكم صلاة الجماعة وأدلتهم (Score: 0.2480)

⚠️ No hadiths found for this query either
Note: Reference extraction may need debugging


In [2]:
# Debug: Check what references are being extracted from fatwas
# (Run cells 6 and 8 first to get linked_result2)
import sys
sys.path.insert(0, '..')

print("=" * 70)
print("DEBUG: Checking reference extraction from fatwas")
print("=" * 70)

from backend.rag_service.data_ingestion.reference_extractor import ReferenceExtractor

extractor = ReferenceExtractor()

# Check if we have results from previous cell
try:
    if linked_result2 and linked_result2.primary_docs:
        print(f"\nAnalyzing {len(linked_result2.primary_docs)} fatwas for hadith references...\n")
        
        for i, fatwa in enumerate(linked_result2.primary_docs, 1):
            print(f"[Fatwa {i}] {fatwa.title[:60]}...")
            print(f"Content length: {len(fatwa.content)} chars")
            
            # Extract hadith references
            hadith_refs = extractor.extract_hadiths(fatwa.content)
            
            if hadith_refs:
                print(f"✓ Found {len(hadith_refs)} hadith references:")
                for ref in hadith_refs:
                    print(f"  - Text: {ref.get('text', '')[:80]}...")
                    print(f"    Narrator: {ref.get('narrator', 'N/A')}")
                    print(f"    Source: {ref.get('source', 'N/A')}")
                    print(f"    Query: {ref.get('query', 'N/A')[:60]}...")
            else:
                print("  ❌ No hadith references extracted")
                # Show some of the content to see what patterns might be there
                content_preview = fatwa.content[:400].replace('\n', ' ')
                print(f"  Content preview: {content_preview}...")
            
            print()
    else:
        print("❌ No linked_result2 available. Run cell 8 first!")
        
except NameError:
    print("❌ linked_result2 not defined. Run cell 8 first!")

print("\n" + "=" * 70)
print("Analysis: Reference extraction results")
print("=" * 70)
print("\nThe reference extractor looks for Arabic patterns like:")
print("  - 'رواه البخاري' (narrated by Bukhari)")
print("  - 'قال رسول الله صلى الله عليه وسلم' (The Prophet said)")
print("  - 'عن أبي هريرة' (from Abu Huraira)")
print("\n📝 Note: If fatwas don't use these exact patterns, hadiths won't be linked.")
print("\n💡 Alternative approach: Use semantic search to find topically relevant")
print("   hadiths based on fatwa content, not just explicit references.")


DEBUG: Checking reference extraction from fatwas
❌ linked_result2 not defined. Run cell 8 first!

Analysis: Reference extraction results

The reference extractor looks for Arabic patterns like:
  - 'رواه البخاري' (narrated by Bukhari)
  - 'قال رسول الله صلى الله عليه وسلم' (The Prophet said)
  - 'عن أبي هريرة' (from Abu Huraira)

📝 Note: If fatwas don't use these exact patterns, hadiths won't be linked.

💡 Alternative approach: Use semantic search to find topically relevant
   hadiths based on fatwa content, not just explicit references.


# Test Enhanced Linked Retrieval with Semantic Fallback

Now testing the improved hadith linking that uses **semantic search** when pattern extraction doesn't find explicit references.

In [3]:
# Test enhanced linked retrieval with semantic fallback
import sys
sys.path.insert(0, '..')

# Force reload to get latest code
if 'backend.rag_service.core.linked_retrieval' in sys.modules:
    del sys.modules['backend.rag_service.core.linked_retrieval']
if 'backend.rag_service.core.rag_engine' in sys.modules:
    del sys.modules['backend.rag_service.core.rag_engine']

from backend.rag_service.core.rag_engine import RAGEngine, RAGConfig

print("=" * 70)
print("Testing ENHANCED Linked Retrieval with Semantic Fallback")
print("=" * 70)

query = "ما حكم الزكاة"
print(f"\nQuery: {query}")
print("Expected: Should now find hadiths about zakat/charity\n")

config = RAGConfig(
    top_k=3,
    use_linked_retrieval=True,
    hadith_collection="hadiths",
    max_hadiths_per_fatwa=2,
    include_citations=True,
)

engine = RAGEngine(config=config, provider_name="gemini")

if engine.linked_retriever:
    # Test with semantic fallback enabled (default)
    result = engine.linked_retriever.retrieve_with_links(
        query=query,
        top_k_fatwas=3,
        top_k_hadiths_per_fatwa=2,
        use_semantic_fallback=True,  # Explicitly enable
    )
    
    print(f"✓ Retrieved {len(result.primary_docs)} fatwas")
    print(f"✓ Retrieved {len(result.linked_hadiths)} linked hadiths")
    print(f"✓ Total documents: {result.total_documents}\n")
    
    # Show fatwas
    print("-" * 70)
    print("FATWAS:")
    print("-" * 70)
    for i, fatwa in enumerate(result.primary_docs, 1):
        print(f"\n[{i}] {fatwa.title[:80]}")
        print(f"    Score: {fatwa.score:.4f}")
    
    # Show linked hadiths
    if result.linked_hadiths:
        print("\n" + "=" * 70)
        print("✅ LINKED HADITHS (via Semantic Search):")
        print("=" * 70)
        for i, hadith in enumerate(result.linked_hadiths, 1):
            print(f"\n[Hadith {i}]")
            print(f"  Source: {hadith.metadata.get('source', 'Unknown')}")
            print(f"  Book: {hadith.metadata.get('book', 'Unknown')}")
            print(f"  Narrator: {hadith.metadata.get('narrator', 'Unknown')[:60]}...")
            print(f"  Score: {hadith.score:.4f}")
            print(f"  Arabic: {hadith.content[:200]}...")
            if hadith.metadata.get('english_translation'):
                print(f"  English: {hadith.metadata.get('english_translation')[:150]}...")
        
        print("\n" + "=" * 70)
        print("✅ SUCCESS! Semantic fallback is working!")
        print("=" * 70)
        print(f"\nRetrieved {len(result.linked_hadiths)} topically relevant hadiths")
        print("These hadiths are about the same topic (zakat) even though")
        print("the fatwas don't explicitly cite them with 'رواه البخاري' patterns.")
    else:
        print("\n❌ Still no hadiths found - check logs for errors")
else:
    print("❌ Linked retriever not initialized!")

Testing ENHANCED Linked Retrieval with Semantic Fallback

Query: ما حكم الزكاة
Expected: Should now find hadiths about zakat/charity



d:\_hsproject\nlp project\backend\..\backend\rag_service\core\retriever.py:105: UserWarning: Api key is used with an insecure connection.
  self._qdrant_client = QdrantClient(
2025-12-28 16:32:45.099 | INFO     | backend.rag_service.core.retriever:initialize:112 - Connected to Qdrant at localhost:6333
2025-12-28 16:33:03.509 | INFO     | backend.rag_service.core.retriever:initialize:123 - Loaded embedding model: Omartificial-Intelligence-Space/GATE-AraBert-v1
2025-12-28 16:33:07.310 | INFO     | backend.rag_service.core.retriever:initialize:134 - Loaded re-ranker: D:\_hsproject\nlp project\testing\model\Namaa-ARA-Reranker-V1
2025-12-28 16:33:08.217 | INFO     | backend.rag_service.core.linked_retrieval:retrieve_with_links:119 - Retrieved 3 fatwas for query
2025-12-28 16:33:08.220 | INFO     | backend.rag_service.core.linked_retrieval:retrieve_with_links:156 - No hadith references found via patterns, using semantic search fallback
d:\_hsproject\nlp project\backend\..\backend\rag_service

✓ Retrieved 3 fatwas
✓ Retrieved 2 linked hadiths
✓ Total documents: 5

----------------------------------------------------------------------
FATWAS:
----------------------------------------------------------------------

[1] زكاة الأسهم المشتراة بقصد الانتفاع بأرباحها
    Score: 0.0333

[2] زكاة من عنده مال وعليه ديون تستغرقه
    Score: 0.0208

[3] المال الذي تجب فيه الزكاة
    Score: 0.0144

✅ LINKED HADITHS (via Semantic Search):

[Hadith 1]
  Source: Sahih Bukhari
  Book: Unknown
  Narrator: Narrated Abu Huraira:...
  Score: 0.0019
  Arabic: حَدَّثَنَا مُسْلِمُ بْنُ إِبْرَاهِيمَ، حَدَّثَنَا وُهَيْبٌ، حَدَّثَنَا ابْنُ طَاوُسٍ، عَنْ أَبِيهِ، عَنْ أَبِي هُرَيْرَةَ ـ رضى الله عنه ـ عَنِ النَّبِيِّ صلى الله عليه وسلم قَالَ ‏
"‏ فَتَحَ اللَّهُ ...

[Hadith 2]
  Source: Sunan al-Darimi
  Book: Unknown
  Narrator: Unknown...
  Score: 0.0019
  Arabic: حَدَّثَنَا أَبُو بَكْرِ بْنُ أَبِي شَيْبَةَ ، قَالَ : حَدَّثَنَا هُشَيْمٌ ، عَنْ مُغِيرَةَ ، عَنْ شِبَاكٍ ، عَنْ إِبْرَاهِيمَ ، قَالَ :" لَا

In [2]:
# Test with model sharing fix - should be faster now
import sys
sys.path.insert(0, '..')

if 'backend.rag_service.core.linked_retrieval' in sys.modules:
    del sys.modules['backend.rag_service.core.linked_retrieval']
if 'backend.rag_service.core.rag_engine' in sys.modules:
    del sys.modules['backend.rag_service.core.rag_engine']

from backend.rag_service.core.rag_engine import RAGEngine, RAGConfig
import time

print("=" * 70)
print("Testing with Model Sharing Fix (should be faster)")
print("=" * 70)

start = time.time()

config = RAGConfig(
    top_k=3,
    use_linked_retrieval=True,
    hadith_collection="hadiths",
    max_hadiths_per_fatwa=2,
)

engine = RAGEngine(config=config, provider_name="gemini")

if engine.linked_retriever:
    result = engine.linked_retriever.retrieve_with_links(
        query="ما حكم صلاة الجماعة",
        top_k_fatwas=3,
        top_k_hadiths_per_fatwa=2,
        use_semantic_fallback=True,
    )
    
    elapsed = time.time() - start
    
    print(f"\n✓ Retrieved {len(result.primary_docs)} fatwas")
    print(f"✓ Retrieved {len(result.linked_hadiths)} linked hadiths")
    print(f"✓ Time: {elapsed:.1f}s (should be ~20s, not ~35s)")
    
    if result.linked_hadiths:
        print("\n" + "=" * 70)
        print("Linked Hadiths:")
        print("=" * 70)
        for i, h in enumerate(result.linked_hadiths, 1):
            print(f"\n[{i}] {h.metadata.get('source', 'Unknown')} - Book: {h.metadata.get('book', 'Unknown')}")
            print(f"    Score: {h.score:.4f}")
            print(f"    {h.content[:150]}...")
        
        print("\n✅ Both semantic fallback AND model sharing working!")
    else:
        print("\n⚠️ No hadiths found")
else:
    print("❌ Linked retriever not initialized!")

Testing with Model Sharing Fix (should be faster)


d:\_hsproject\nlp project\backend\..\backend\rag_service\core\retriever.py:105: UserWarning: Api key is used with an insecure connection.
  self._qdrant_client = QdrantClient(
2025-12-28 16:37:00.571 | INFO     | backend.rag_service.core.retriever:initialize:112 - Connected to Qdrant at localhost:6333
2025-12-28 16:37:19.288 | INFO     | backend.rag_service.core.retriever:initialize:123 - Loaded embedding model: Omartificial-Intelligence-Space/GATE-AraBert-v1
2025-12-28 16:37:23.556 | INFO     | backend.rag_service.core.retriever:initialize:134 - Loaded re-ranker: D:\_hsproject\nlp project\testing\model\Namaa-ARA-Reranker-V1
2025-12-28 16:37:31.071 | INFO     | backend.rag_service.core.linked_retrieval:retrieve_with_links:119 - Retrieved 3 fatwas for query
2025-12-28 16:37:31.074 | INFO     | backend.rag_service.core.linked_retrieval:retrieve_with_links:129 - Found 1 hadith references via pattern extraction
d:\_hsproject\nlp project\backend\..\backend\rag_service\core\retriever.py:105:


✓ Retrieved 3 fatwas
✓ Retrieved 0 linked hadiths
✓ Time: 42.9s (should be ~20s, not ~35s)

⚠️ No hadiths found


## Final Test: All Fixes Applied

Testing with:
- ✅ Semantic fallback for hadith linking
- ✅ Model sharing (should load models once, not twice)
- ✅ Lower threshold (0.15 instead of 0.6) for both pattern and semantic retrieval

In [ ]:
# Final comprehensive test with all fixes
import sys
sys.path.insert(0, '..')

if 'backend.rag_service.core.linked_retrieval' in sys.modules:
    del sys.modules['backend.rag_service.core.linked_retrieval']
if 'backend.rag_service.core.rag_engine' in sys.modules:
    del sys.modules['backend.rag_service.core.rag_engine']

from backend.rag_service.core.rag_engine import RAGEngine, RAGConfig
import time

print("=" * 70)
print("FINAL TEST: Semantic Fallback + Model Sharing + Lower Threshold")
print("=" * 70)

# Test 1: Query with no explicit hadith references (should use semantic fallback)
print("\n[Test 1] Query: 'ما حكم الزكاة' (zakat ruling)")
print("Expected: Semantic fallback finds hadiths about zakat\n")

start1 = time.time()
config1 = RAGConfig(top_k=3, use_linked_retrieval=True, hadith_collection="hadiths")
engine1 = RAGEngine(config=config1, provider_name="gemini")

if engine1.linked_retriever:
    result1 = engine1.linked_retriever.retrieve_with_links(
        query="ما حكم الزكاة",
        top_k_fatwas=3,
        top_k_hadiths_per_fatwa=2,
        use_semantic_fallback=True,
    )
    elapsed1 = time.time() - start1
    
    print(f"✓ Fatwas: {len(result1.primary_docs)}")
    print(f"✓ Hadiths: {len(result1.linked_hadiths)}")
    print(f"✓ Time: {elapsed1:.1f}s")
    
    if result1.linked_hadiths:
        print("\n📚 Hadiths found:")
        for i, h in enumerate(result1.linked_hadiths, 1):
            print(f"  [{i}] {h.metadata.get('source')} - Score: {h.score:.4f}")
            print(f"      {h.content[:100]}...")

print("\n" + "=" * 70)

# Test 2: Query that might have explicit references (test pattern + lower threshold)
print("\n[Test 2] Query: 'ما حكم صلاة الجماعة' (congregational prayer)")
print("Expected: Pattern extraction OR semantic fallback\n")

start2 = time.time()
config2 = RAGConfig(top_k=3, use_linked_retrieval=True, hadith_collection="hadiths")
engine2 = RAGEngine(config=config2, provider_name="gemini")

if engine2.linked_retriever:
    result2 = engine2.linked_retriever.retrieve_with_links(
        query="ما حكم صلاة الجماعة",
        top_k_fatwas=3,
        top_k_hadiths_per_fatwa=2,
        use_semantic_fallback=True,
    )
    elapsed2 = time.time() - start2
    
    print(f"✓ Fatwas: {len(result2.primary_docs)}")
    print(f"✓ Hadiths: {len(result2.linked_hadiths)}")
    print(f"✓ Time: {elapsed2:.1f}s (should be faster with model sharing!)")
    
    if result2.linked_hadiths:
        print("\n📚 Hadiths found:")
        for i, h in enumerate(result2.linked_hadiths, 1):
            print(f"  [{i}] {h.metadata.get('source')} - Score: {h.score:.4f}")
            print(f"      {h.content[:100]}...")

print("\n" + "=" * 70)
print("SUMMARY")
print("=" * 70)
total_hadiths = len(result1.linked_hadiths) + len(result2.linked_hadiths)
if total_hadiths > 0:
    print(f"✅ SUCCESS! Found {total_hadiths} hadiths total")
    print(f"✅ Model sharing: Test 2 faster than Test 1? {elapsed2 < elapsed1}")
    print("\nEnhancements working:")
    print("  • Semantic fallback for queries without explicit references")
    print("  • Lower threshold (0.15) allows more relevant results")
    print("  • Model sharing reduces initialization time")
else:
    print("❌ No hadiths found - check logs for issues")

FINAL TEST: Semantic Fallback + Model Sharing + Lower Threshold

[Test 1] Query: 'ما حكم الزكاة' (zakat ruling)
Expected: Semantic fallback finds hadiths about zakat



d:\_hsproject\nlp project\backend\..\backend\rag_service\core\retriever.py:105: UserWarning: Api key is used with an insecure connection.
  self._qdrant_client = QdrantClient(
2025-12-28 16:39:11.675 | INFO     | backend.rag_service.core.retriever:initialize:112 - Connected to Qdrant at localhost:6333
2025-12-28 16:39:27.405 | INFO     | backend.rag_service.core.retriever:initialize:123 - Loaded embedding model: Omartificial-Intelligence-Space/GATE-AraBert-v1
2025-12-28 16:39:31.167 | INFO     | backend.rag_service.core.retriever:initialize:134 - Loaded re-ranker: D:\_hsproject\nlp project\testing\model\Namaa-ARA-Reranker-V1
2025-12-28 16:39:31.953 | INFO     | backend.rag_service.core.linked_retrieval:retrieve_with_links:119 - Retrieved 3 fatwas for query
2025-12-28 16:39:31.955 | INFO     | backend.rag_service.core.linked_retrieval:_share_models_if_needed:82 - Sharing embedding model and reranker with hadith retriever
2025-12-28 16:39:31.958 | INFO     | backend.rag_service.core.link

✓ Fatwas: 3
✓ Hadiths: 2
✓ Time: 22.9s

📚 Hadiths found:
  [1] Sahih Bukhari - Score: 0.0019
      حَدَّثَنَا مُسْلِمُ بْنُ إِبْرَاهِيمَ، حَدَّثَنَا وُهَيْبٌ، حَدَّثَنَا ابْنُ طَاوُسٍ، عَنْ أَبِيهِ، ...
  [2] Sunan al-Darimi - Score: 0.0019
      حَدَّثَنَا أَبُو بَكْرِ بْنُ أَبِي شَيْبَةَ ، قَالَ : حَدَّثَنَا هُشَيْمٌ ، عَنْ مُغِيرَةَ ، عَنْ شِ...


[Test 2] Query: 'ما حكم صلاة الجماعة' (congregational prayer)
Expected: Pattern extraction OR semantic fallback



2025-12-28 16:39:33.017 | INFO     | backend.rag_service.core.retriever:initialize:112 - Connected to Qdrant at localhost:6333
2025-12-28 16:39:36.632 | INFO     | backend.rag_service.core.retriever:initialize:123 - Loaded embedding model: Omartificial-Intelligence-Space/GATE-AraBert-v1
2025-12-28 16:39:40.144 | INFO     | backend.rag_service.core.retriever:initialize:134 - Loaded re-ranker: D:\_hsproject\nlp project\testing\model\Namaa-ARA-Reranker-V1


## Test Multiple Prompts with Single Engine

Testing 5 different queries with **one** engine instance (no VRAM waste from reloading models)

In [1]:
# Test 5 different prompts with a SINGLE engine instance (saves VRAM!)
import sys
sys.path.insert(0, '..')

if 'backend.rag_service.core.linked_retrieval' in sys.modules:
    del sys.modules['backend.rag_service.core.linked_retrieval']
if 'backend.rag_service.core.rag_engine' in sys.modules:
    del sys.modules['backend.rag_service.core.rag_engine']

from backend.rag_service.core.rag_engine import RAGEngine, RAGConfig
import time

print("=" * 70)
print("Testing 5 Prompts with Single Engine Instance")
print("=" * 70)

# Initialize engine ONCE
print("\n[Initialization] Creating engine (models load once)...\n")
start_init = time.time()

config = RAGConfig(
    top_k=3,
    use_linked_retrieval=True,
    hadith_collection="hadiths",
    max_hadiths_per_fatwa=2,
    include_citations=True,
)

engine = RAGEngine(config=config, provider_name="gemini")
elapsed_init = time.time() - start_init
print(f"✓ Engine initialized in {elapsed_init:.1f}s\n")


Testing 5 Prompts with Single Engine Instance

[Initialization] Creating engine (models load once)...

✓ Engine initialized in 0.0s



In [2]:

# Test queries
test_queries = [
    "ما حكم الزكاة",                    # 1. Zakat ruling
    "ما حكم صلاة الجماعة",              # 2. Congregational prayer
    "حكم الصيام في رمضان",              # 3. Fasting in Ramadan
    "ما حكم الحج",                      # 4. Hajj ruling
    "حكم قراءة القرآن",                 # 5. Quran recitation
]

query_names = [
    "Zakat (charity)",
    "Salat al-Jama'ah (congregational prayer)",
    "Fasting in Ramadan",
    "Hajj (pilgrimage)",
    "Quran recitation",
]

print("=" * 70)
print("Running 5 queries (reusing same engine)...")
print("=" * 70)

results = []

for i, (query, name) in enumerate(zip(test_queries, query_names), 1):
    print(f"\n[Query {i}/5] {name}")
    print(f"Arabic: {query}")
    
    start = time.time()
    
    if engine.linked_retriever:
        result = engine.linked_retriever.retrieve_with_links(
            query=query,
            top_k_fatwas=3,
            top_k_hadiths_per_fatwa=2,
            use_semantic_fallback=True,
        )
        elapsed = time.time() - start
        
        results.append({
            'query': query,
            'name': name,
            'fatwas': len(result.primary_docs),
            'hadiths': len(result.linked_hadiths),
            'time': elapsed,
            'result': result,
        })
        
        print(f"  ✓ Fatwas: {len(result.primary_docs)}")
        print(f"  ✓ Hadiths: {len(result.linked_hadiths)}")
        print(f"  ✓ Time: {elapsed:.2f}s")
        
        if result.linked_hadiths:
            print(f"  📚 Hadith sources: {', '.join(set(h.metadata.get('source', 'Unknown') for h in result.linked_hadiths))}")

print("\n" + "=" * 70)
print("SUMMARY")
print("=" * 70)

total_fatwas = sum(r['fatwas'] for r in results)
total_hadiths = sum(r['hadiths'] for r in results)
total_time = sum(r['time'] for r in results)
avg_time = total_time / len(results)

print(f"\n📊 Results:")
print(f"  Total fatwas retrieved: {total_fatwas}")
print(f"  Total hadiths retrieved: {total_hadiths}")
print(f"  Total query time: {total_time:.1f}s")
print(f"  Average per query: {avg_time:.2f}s")
print(f"  Initialization time: {elapsed_init:.1f}s (one-time)")

print(f"\n✅ SUCCESS! Tested 5 queries with single engine")
print(f"   Models loaded ONCE, not 5 times (VRAM efficient!)")

# Show detailed results table
print("\n" + "=" * 70)
print("Detailed Results")
print("=" * 70)
print(f"\n{'Query':<40} {'Fatwas':<8} {'Hadiths':<8} {'Time':<8}")
print("-" * 70)
for r in results:
    print(f"{r['name']:<40} {r['fatwas']:<8} {r['hadiths']:<8} {r['time']:.2f}s")

# Check if any queries found hadiths
queries_with_hadiths = sum(1 for r in results if r['hadiths'] > 0)
print(f"\n✓ Queries that found hadiths: {queries_with_hadiths}/{len(results)}")

if queries_with_hadiths > 0:
    print("✅ Semantic fallback is working across different topics!")
else:
    print("⚠️ No hadiths found - check threshold or semantic search")

Running 5 queries (reusing same engine)...

[Query 1/5] Zakat (charity)
Arabic: ما حكم الزكاة


d:\_hsproject\nlp project\backend\..\backend\rag_service\core\retriever.py:105: UserWarning: Api key is used with an insecure connection.
  self._qdrant_client = QdrantClient(
2025-12-28 16:47:06.355 | INFO     | backend.rag_service.core.retriever:initialize:112 - Connected to Qdrant at localhost:6333
2025-12-28 16:47:22.088 | INFO     | backend.rag_service.core.retriever:initialize:123 - Loaded embedding model: Omartificial-Intelligence-Space/GATE-AraBert-v1
2025-12-28 16:47:25.649 | INFO     | backend.rag_service.core.retriever:initialize:134 - Loaded re-ranker: D:\_hsproject\nlp project\testing\model\Namaa-ARA-Reranker-V1
2025-12-28 16:47:26.374 | INFO     | backend.rag_service.core.linked_retrieval:retrieve_with_links:119 - Retrieved 3 fatwas for query
2025-12-28 16:47:26.376 | INFO     | backend.rag_service.core.linked_retrieval:_share_models_if_needed:82 - Sharing embedding model and reranker with hadith retriever
2025-12-28 16:47:26.379 | INFO     | backend.rag_service.core.link

  ✓ Fatwas: 3
  ✓ Hadiths: 2
  ✓ Time: 22.60s
  📚 Hadith sources: Sahih Bukhari, Sunan al-Darimi

[Query 2/5] Salat al-Jama'ah (congregational prayer)
Arabic: ما حكم صلاة الجماعة


2025-12-28 16:47:33.482 | INFO     | backend.rag_service.core.linked_retrieval:retrieve_with_links:119 - Retrieved 3 fatwas for query
2025-12-28 16:47:33.483 | INFO     | backend.rag_service.core.linked_retrieval:retrieve_with_links:133 - Found 1 hadith references via pattern extraction
2025-12-28 16:47:33.594 | INFO     | backend.rag_service.core.linked_retrieval:retrieve_with_links:156 - Retrieved 1 unique hadiths via patterns


  ✓ Fatwas: 3
  ✓ Hadiths: 1
  ✓ Time: 7.04s
  📚 Hadith sources: Sunan al-Tirmidhi

[Query 3/5] Fasting in Ramadan
Arabic: حكم الصيام في رمضان


...

  ✓ Fatwas: 3
  ✓ Hadiths: 2
  ✓ Time: 0.65s
  📚 Hadith sources: Sahih Bukhari, Sunan al-Tirmidhi

[Query 4/5] Hajj (pilgrimage)
Arabic: ما حكم الحج


2025-12-28 16:47:34.937 | INFO     | backend.rag_service.core.linked_retrieval:retrieve_with_links:119 - Retrieved 3 fatwas for query
2025-12-28 16:47:34.938 | INFO     | backend.rag_service.core.linked_retrieval:retrieve_with_links:160 - No hadith references found via patterns, using semantic search fallback
2025-12-28 16:47:35.177 | INFO     | backend.rag_service.core.linked_retrieval:retrieve_with_links:166 - Retrieved 2 hadiths via semantic search


  ✓ Fatwas: 3
  ✓ Hadiths: 2
  ✓ Time: 0.93s
  📚 Hadith sources: Sahih Bukhari, Sunan al-Tirmidhi

[Query 5/5] Quran recitation
Arabic: حكم قراءة القرآن


2025-12-28 16:47:35.598 | INFO     | backend.rag_service.core.linked_retrieval:retrieve_with_links:119 - Retrieved 3 fatwas for query
2025-12-28 16:47:35.601 | INFO     | backend.rag_service.core.linked_retrieval:retrieve_with_links:160 - No hadith references found via patterns, using semantic search fallback
2025-12-28 16:47:35.775 | INFO     | backend.rag_service.core.linked_retrieval:retrieve_with_links:166 - Retrieved 2 hadiths via semantic search


  ✓ Fatwas: 3
  ✓ Hadiths: 2
  ✓ Time: 0.60s
  📚 Hadith sources: Sahih Bukhari

SUMMARY

📊 Results:
  Total fatwas retrieved: 15
  Total hadiths retrieved: 9
  Total query time: 31.8s
  Average per query: 6.36s
  Initialization time: 0.0s (one-time)

✅ SUCCESS! Tested 5 queries with single engine
   Models loaded ONCE, not 5 times (VRAM efficient!)

Detailed Results

Query                                    Fatwas   Hadiths  Time    
----------------------------------------------------------------------
Zakat (charity)                          3        2        22.60s
Salat al-Jama'ah (congregational prayer) 3        1        7.04s
Fasting in Ramadan                       3        2        0.65s
Hajj (pilgrimage)                        3        2        0.93s
Quran recitation                         3        2        0.60s

✓ Queries that found hadiths: 5/5
✅ Semantic fallback is working across different topics!


## Test: Models Load During Initialization (Not First Query)

Testing the fix where models load when creating the engine, not lazily on first query.

In [7]:
# Test that models load during engine initialization, not first query
import sys
sys.path.insert(0, '..')

if 'backend.rag_service.core.linked_retrieval' in sys.modules:
    del sys.modules['backend.rag_service.core.linked_retrieval']
if 'backend.rag_service.core.rag_engine' in sys.modules:
    del sys.modules['backend.rag_service.core.rag_engine']

from backend.rag_service.core.rag_engine import RAGEngine, RAGConfig
import time

print("=" * 70)
print("Testing: Models Load During Initialization")
print("=" * 70)

# Time the initialization
print("\n[Phase 1] Creating RAG engine...")
print("Expected: Models load HERE (15-20s)\n")
start_init = time.time()

config = RAGConfig(
    top_k=3,
    use_linked_retrieval=True,
    hadith_collection="hadiths",
    max_hadiths_per_fatwa=2,
)

engine = RAGEngine(config=config, provider_name="gemini")
elapsed_init = time.time() - start_init

print(f"\n✓ Engine initialized in {elapsed_init:.1f}s")
print(f"   Expected: ~15-20s (models loaded)")
print(f"   Actual: {'✅ GOOD' if elapsed_init > 10 else '⚠️ TOO FAST - models may not have loaded'}\n")

# Now run first query - should be fast!
print("=" * 70)
print("[Phase 2] Running first query...")
print("Expected: Should be FAST now (<2s)\n")

start_query = time.time()

if engine.linked_retriever:
    result = engine.linked_retriever.retrieve_with_links(
        query="ما حكم الزكاة",
        top_k_fatwas=3,
        top_k_hadiths_per_fatwa=2,
        use_semantic_fallback=True,
    )
    elapsed_query = time.time() - start_query
    
    print(f"✓ Query completed in {elapsed_query:.2f}s")
    print(f"   Expected: <2s (models already loaded)")
    print(f"   Actual: {'✅ EXCELLENT' if elapsed_query < 2 else '⚠️ SLOWER than expected'}")
    print(f"\n✓ Retrieved {len(result.primary_docs)} fatwas, {len(result.linked_hadiths)} hadiths")

print("\n" + "=" * 70)
print("RESULT")
print("=" * 70)

if elapsed_init > 10 and elapsed_query < 2:
    print("✅ SUCCESS! Models load during initialization, not on first query")
    print(f"   Init: {elapsed_init:.1f}s (models loaded)")
    print(f"   Query: {elapsed_query:.2f}s (fast!)")
elif elapsed_init < 5:
    print("⚠️ Models may not have loaded during initialization")
    print(f"   Init was too fast: {elapsed_init:.1f}s")
    print(f"   First query: {elapsed_query:.2f}s")
else:
    print("✓ Models loaded, but query could be faster")
    print(f"   Init: {elapsed_init:.1f}s")
    print(f"   Query: {elapsed_query:.2f}s")

2025-12-28 17:09:24.971 | INFO     | backend.rag_service.core.rag_engine:__init__:145 - Initializing retrievers and loading models...


Testing: Models Load During Initialization

[Phase 1] Creating RAG engine...
Expected: Models load HERE (15-20s)



2025-12-28 17:09:25.843 | INFO     | backend.rag_service.core.retriever:initialize:112 - Connected to Qdrant at localhost:6333
2025-12-28 17:09:30.180 | INFO     | backend.rag_service.core.retriever:initialize:123 - Loaded embedding model: Omartificial-Intelligence-Space/GATE-AraBert-v1
2025-12-28 17:09:30.181 | INFO     | backend.rag_service.core.linked_retrieval:_share_models_if_needed:82 - Sharing embedding model and reranker with hadith retriever
2025-12-28 17:09:30.181 | INFO     | backend.rag_service.core.rag_engine:__init__:152 - RAG Engine initialization complete
2025-12-28 17:09:30.233 | INFO     | backend.rag_service.core.linked_retrieval:retrieve_with_links:119 - Retrieved 3 fatwas for query
2025-12-28 17:09:30.236 | INFO     | backend.rag_service.core.linked_retrieval:retrieve_with_links:160 - No hadith references found via patterns, using semantic search fallback
2025-12-28 17:09:30.282 | INFO     | backend.rag_service.core.linked_retrieval:retrieve_with_links:166 - Retrie


✓ Engine initialized in 5.2s
   Expected: ~15-20s (models loaded)
   Actual: ⚠️ TOO FAST - models may not have loaded

[Phase 2] Running first query...
Expected: Should be FAST now (<2s)

✓ Query completed in 0.10s
   Expected: <2s (models already loaded)
   Actual: ✅ EXCELLENT

✓ Retrieved 3 fatwas, 2 hadiths

RESULT
✓ Models loaded, but query could be faster
   Init: 5.2s
   Query: 0.10s


## Detailed Results Review

Displaying full details for each query to verify hadith quality and citations

In [8]:
# Display detailed results for each query
print("=" * 80)
print("DETAILED QUERY RESULTS - Quality Review")
print("=" * 80)

for idx, r in enumerate(results, 1):
    print(f"\n{'=' * 80}")
    print(f"QUERY {idx}/5: {r['name']}")
    print(f"Arabic: {r['query']}")
    print("=" * 80)
    
    result = r['result']
    
    # 1. FATWAS
    print(f"\n📚 FATWAS RETRIEVED ({len(result.primary_docs)}):")
    print("-" * 80)
    for i, fatwa in enumerate(result.primary_docs, 1):
        print(f"\n[Fatwa {i}]")
        print(f"  Title: {fatwa.title[:100] if fatwa.title else fatwa.fatwa_id}")
        print(f"  Source: {fatwa.source}")
        print(f"  Score: {fatwa.score:.4f}")
        print(f"  Content preview: {fatwa.content[:300]}...")
    
    # 2. HADITHS
    if result.linked_hadiths:
        print(f"\n\n📖 HADITHS RETRIEVED ({len(result.linked_hadiths)}):")
        print("-" * 80)
        for i, hadith in enumerate(result.linked_hadiths, 1):
            print(f"\n[Hadith {i}]")
            print(f"  Source: {hadith.metadata.get('source', 'Unknown')}")
            print(f"  Book: {hadith.metadata.get('book', 'Unknown')}")
            print(f"  Chapter: {hadith.metadata.get('chapter', 'Unknown')}")
            print(f"  Number: {hadith.metadata.get('number', 'Unknown')}")
            print(f"  Narrator: {hadith.metadata.get('narrator', 'Unknown')[:80]}...")
            print(f"  Score: {hadith.score:.4f}")
            print(f"\n  📝 Arabic Text:")
            print(f"  {hadith.content[:400]}")
            if len(hadith.content) > 400:
                print(f"  ... (continued)")
            
            if hadith.metadata.get('english_translation'):
                print(f"\n  🌐 English Translation:")
                print(f"  {hadith.metadata.get('english_translation')[:300]}")
                if len(hadith.metadata.get('english_translation', '')) > 300:
                    print(f"  ... (continued)")
    else:
        print(f"\n\n📖 HADITHS: None found")
    
    # 3. CITATIONS
    print(f"\n\n📑 CITATIONS:")
    print("-" * 80)
    all_docs = result.all_documents
    citations = engine.citation_generator.generate_citations(all_docs)
    
    if citations:
        for i, citation in enumerate(citations, 1):
            print(f"\n[{i}] {citation.get('formatted', citation.get('source', 'Unknown'))}")
    else:
        print("No citations generated")
    
    # 4. CONTEXT PREVIEW
    print(f"\n\n📄 FORMATTED CONTEXT:")
    print("-" * 80)
    formatted_context = engine.context_manager.format_context(all_docs)
    print(f"Total documents: {len(formatted_context.documents_used)}")
    print(f"Context length: {len(formatted_context.text)} chars (~{len(formatted_context.text) // 4} tokens)")
    print(f"Truncated: {formatted_context.truncated}")
    print(f"\nContext preview (first 500 chars):")
    print(formatted_context.text[:500])
    print("...")
    
    print(f"\n{'=' * 80}\n")

print("\n" + "=" * 80)
print("✅ Review complete! Check hadith relevance and citation quality above.")
print("=" * 80)

DETAILED QUERY RESULTS - Quality Review

QUERY 1/5: Zakat (charity)
Arabic: ما حكم الزكاة

📚 FATWAS RETRIEVED (3):
--------------------------------------------------------------------------------

[Fatwa 1]
  Title: الأصناف التي تجب فيها الزكاة
  Source: إسلام ويب
  Score: 0.7280
  Content preview: السؤال: ما هي الأشياء والممتلكات الشخصية الواجب الزكاة عنها سنويا وأرجو أن تكون بالتحديد السهل  وجزاكم الله خيرا.

الجواب: الحمد لله، والصلاة والسلام على رسول الله، وعلى آله وصحبه، أما بعد: فتجب الزكاة سنويا في النقدين: الذهب والفضة ويلحق بهما الأوراق النقدية المتداولة بين أيدي الناس الآن، وعروض الت...

[Fatwa 2]
  Title: حكم الماس الزينة
  Source: binbaz
  Score: 0.7262
  Content preview: السؤال: س: الماس الذي يستعمل للزينة واللبس هل فيه زكاة؟

الجواب: حكم الماس الزينة
                
                                    
                         س: الماس الذي يستعمل للزينة واللبس هل فيه زكاة؟

                    
                                
                
                   ...

[F